<a href="https://colab.research.google.com/github/irenfonseca/ventilation_certificate/blob/main/ventilation_cert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ventilation certificate

In [ ]:
# Data import 
!pip install seaborn
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import TensorBoard

print(tf.__version__)

### Data preparation

#### Getting data from google drive
The data are preprocessed for nominal values with one-hot

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open('data').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
dataset = pd.DataFrame.from_records(rows)
dataset = dataset.astype(int)

print(dataset.head())
print(dataset.shape)

Checking all data are correct



In [ ]:
dataset.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
dtype: int64

#### Separate data into training and testing


In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
train_dataset = train_dataset.astype(int)

test_dataset = dataset.drop(train_dataset.index)
test_dataset = test_dataset.astype(int)

train_dataset.head()


#### Statistics

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop(10)
train_stats = train_stats.transpose()
train_stats


#### Separate features from tags.



In [ ]:
train_labels = train_dataset.pop(10)
test_labels = test_dataset.pop(10)

#### Data normalization

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)
normed_train_data = normed_train_data.astype(int)
normed_test_data = normed_test_data.astype(int)



### The model

In [ ]:
model = keras.Sequential([
    layers.Dense(100, activation='relu', input_shape=[len(train_dataset.keys())]),
    keras.layers.Dense(70, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')

])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



#### Training 

Using  early stop to avoid overfit

In [ ]:
tensorboardModel = TensorBoard(log_dir='logs/model')
model.fit(normed_train_data, train_labels, epochs=50, validation_split = 0.2, callbacks=[tensorboardModel])



In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/